# ボクセルデータ抽出処理

まずはじめに，Bunny.plyのメッシュデータをファイルから読み込みます．

In [1]:
import open3d as o3d
import numpy as np

filename = "../3rdparty/Open3D/examples/test_data/Bunny.ply"
print("Loading a point cloud from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)
mesh.compute_vertex_normals()

# fit to unit cube
mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
o3d.visualization.draw_geometries([mesh])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading a point cloud from ../3rdparty/Open3D/examples/test_data/Bunny.ply
TriangleMesh with 35947 points and 69451 triangles.


これを実行すると，メッシュデータが表示されます．
なお，上記プログラムの11行目はメッシュデータのバウンディングボックスの一辺の最大値が$1$になるようメッシュデータをスケール変換しています．
次に，このメッシュデータを一辺の長さが$0.05$のボクセルデータに変換します．

In [2]:
print('mesh to voxel')
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=0.05)
o3d.visualization.draw_geometries([voxel_grid])

mesh to voxel


これを実行すると，ボクセルデータが表示されます．
ここでは，メッシュデータ全体を含むバウンディングボックスを等間隔に分割するボクセルデータを用意し，
一つ以上のメッシュと交差するボクセルの値を$1$，他のボクセルの値を$0$とする関数が実装されています．
全てのメッシュと全てのボクセルの交差判定を行うため，処理には少し時間がかかります．
なお，現時点では，メッシュデータをボクセルデータに変換するOpen3Dの関数内ではボクセルデータに色情報を付加する処理が実装されていません．

次に，点群データをボクセルデータに変換する処理を試してみましょう．

In [3]:
print('point cloud to voxel')
pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(np.array(pcd.points).shape[0], 3)))
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.05)
o3d.visualization.draw_geometries([voxel_grid])

point cloud to voxel


ここでは（上記プログラムの3行目で）単純にメッシュの各頂点を点群データとして抽出し，
4行目でランダムな色付けをしています．
上記を実行すると，ボクセルデータが表示されます．
ここでは，一つ以上の点を含むボクセルの値を$1$，他のボクセルの値を$0$とする関数が実装されています．
各点が含まれるボクセルのインデックスは一意に求まるため，メッシュデータを入力とする場合に比べて軽い処理となります．
ここで，各ボクセルの色はそのボクセルが含む全ての点の持つ色の平均値としています．
なお，下記の処理を実行することで，各点が（$1$の値を持つ）ボクセルデータの中に含まれているかどうかを確認することが可能です．

In [4]:
# check if points are within an occupied voxel
queries = np.asarray(pcd.points)
output = voxel_grid.check_if_included(o3d.utility.Vector3dVector(queries))
print(output[:10])

[True, True, True, True, True, True, True, True, True, True]
